## SageMaker Model and Model Registry 
- haimtran 01/06/2023
- [Model Artifact](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-marketplace-develop.html)
- [SageMaker SDK Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html)
- [Retrieve Pre-Built SageMaker Docker Image](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html)
- [SageMaker Docker Registry Paths](https://docs.aws.amazon.com/sagemaker/latest/dg-ecr-paths/ecr-ap-southeast-1.html#pca-ap-southeast-1.title)
- [SageMaker Create Endpoint Config](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)

A deployable model consits of at least 
- Image URI 
- ModelDataURL after training 
- Role 

In [ ]:
# !python -m pip install sagemaker
# !python -m pip install boto3

In [1]:
import sagemaker
import boto3

In [2]:
session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()

In [3]:
session._region_name

'ap-southeast-1'

## Boto3 Check and Existing Model 

In [4]:
sagemaker_client = boto3.client("sagemaker")

In [5]:
sagemaker_client.list_models()

{'Models': [{'ModelName': 'PCACreatedFromNoteBookModel',
   'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model/pcacreatedfromnotebookmodel',
   'CreationTime': datetime.datetime(2023, 6, 1, 5, 26, 13, 396000, tzinfo=tzlocal())},
  {'ModelName': 'PCAModel1aac2da2-df8c-4da8-8097-5cb336dae838',
   'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model/pcamodel1aac2da2-df8c-4da8-8097-5cb336dae838',
   'CreationTime': datetime.datetime(2023, 5, 31, 12, 40, 21, 985000, tzinfo=tzlocal())},
  {'ModelName': 'PCAModel23b02247-812d-4bdc-a980-0e265a880cd5',
   'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model/pcamodel23b02247-812d-4bdc-a980-0e265a880cd5',
   'CreationTime': datetime.datetime(2023, 5, 31, 11, 48, 19, 141000, tzinfo=tzlocal())},
  {'ModelName': 'pca-2023-05-31-08-13-08-496',
   'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model/pca-2023-05-31-08-13-08-496',
   'CreationTime': datetime.datetime(2023, 5, 31, 8, 13, 8, 990000, tzinfo

In [6]:
sagemaker_client.describe_model(
    ModelName="PCAModel1aac2da2-df8c-4da8-8097-5cb336dae838"
)

{'ModelName': 'PCAModel1aac2da2-df8c-4da8-8097-5cb336dae838',
 'PrimaryContainer': {'Image': '475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/pca:1',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz',
  'Environment': {}},
 'ExecutionRoleArn': 'arn:aws:iam::394599967678:role/RoleForDataScientistUserProfile',
 'CreationTime': datetime.datetime(2023, 5, 31, 12, 40, 21, 985000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:ap-southeast-1:394599967678:model/pcamodel1aac2da2-df8c-4da8-8097-5cb336dae838',
 'EnableNetworkIsolation': False,
 'DeploymentRecommendation': {'RecommendationStatus': 'COMPLETED',
  'RealTimeInferenceRecommendations': [{'RecommendationId': 'pcamodel1aac2da2-df8c-4da8-8097-5cb336dae838/RL8o0xf2',
    'InstanceType': 'ml.c6i.xlarge',
    'Environment': {}},
   {'RecommendationId': 'pcamodel1aac2da2-df8c-4da8-8097-5cb336dae838/EcvIWVng',
    'InstanceType'

## SageMaker SDK: Create a Model 

In [7]:
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

In [8]:
image_uri = retrieve(
    framework="pca",
    region=session._region_name, 
    instance_type="ml.m4.xlarge",
    version="0.23-1"
)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 0.23-1.


In [9]:
print(image_uri)

475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/pca:1


In [10]:
model = Model(
    image_uri=image_uri,
    model_data="s3://sagemaker-ap-southeast-1-394599967678/TrainingJobName27a61f71-876d-4d88-a39f-487cd1d1ebd4/output/model.tar.gz",
    role=role,
    name="PCACreatedFromNoteBookModel"
)

In [12]:
model.create(instance_type="ml.m4.xlarge")

Using already existing model: PCACreatedFromNoteBookModel


In [13]:
model.delete_model()

## Deploy a Model as a Endpoint 

[deploy a model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html), and it would take about 5 minutes bofore the endpoint ready 

In [ ]:
model.deploy(
    initial_instance_count=1, 
    instance_type="ml.m4.xlarge"
)

describe the deployed endpoint 

In [ ]:
sagemaker_client.list_endpoints()

In [ ]:
sagemaker_client.describe_endpoint(
    EndpointName="PCACreatedFromNoteBookModel-2023-06-01-05-26-19-579"
)